In [ ]:
# first install python 3.6
!sudo apt-get update -y
!sudo apt-get install python3.6
# change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 1
# select python version
!sudo update-alternatives --config python3
# check python version
!python --version
# install pip for new python 
!sudo apt-get install python3.6-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
# upgrade pip
!sudo apt install python3-pip
!python -m pip install --upgrade pip

In [1]:
X_sentence=list()
Y_sentence=list()
count=0
base_path="dataset"
X_train=list()
Y_train=list()
X_val=list()
Y_val=list()
X_test=list()
Y_test=list()
with open(base_path+"/S1-test.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_test.append(X_sentence)      
                Y_test.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S2-train.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_train.append(X_sentence)      
                Y_train.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S3-val.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_val.append(X_sentence)      
                Y_val.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
# sentences it is a list of sentences. Every sentence is a list of tuple
tokenized_sentences=X_train+X_test+X_val
tokenized_labels=Y_train+Y_test+Y_val

In [2]:
from gensim.models import FastText

model_2=FastText(sentences=tokenized_sentences, vector_size=100, window=10, min_count=5, sg=1, negative=10, alpha=0.01, sample=0.0001)

In [11]:
model_2.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=2)

(3049188, 6991040)

In [12]:
model_2.wv.most_similar("TIG")

[('CMT', 0.9488525986671448),
 ('GTAW', 0.9404873251914978),
 ('Gas', 0.933182418346405),
 ('GMAW', 0.9262470602989197),
 ('Plasma', 0.922127366065979),
 ('wire-arc', 0.9121090769767761),
 ('Tungsten', 0.9078242778778076),
 ('Arc', 0.8962817192077637),
 ('tungsten', 0.8776745796203613),
 ('7075', 0.873650312423706)]

In [ ]:
import importlib
from modelli import wrapper, utils
importlib.reload(wrapper)
from modelli.wrapper import Sequence


lstm=Sequence(embeddings=model_2.wv, initial_vocab=model_2.wv.key_to_index)
lstm.fit_vocab(X_train,Y_train)

: 

: 

In [22]:
len(lstm.p._word_vocab)

4825

In [ ]:
lstm.model.summary()

In [ ]:
from modelli.wrapper import Sequence
import pickle

lstm=Sequence.load("param","weights.h5","preprocessor")
lstm.model.compile( optimizer=lstm.optimizer)
with open('optimizer.pkl', 'rb') as f:
    weight_values = pickle.load(f)
lstm.model.optimizer.set_weights(weight_values)


In [23]:
import importlib
import modelli.trainer
importlib.reload(modelli.trainer)
from modelli.trainer import Trainer



trainer = Trainer(lstm.model, preprocessor=lstm.p)
trainer.train(X_train, Y_train, X_val, Y_val, epochs=1, batch_size=32,verbose=1,shuffle=True)


InvalidArgumentError: indices[18,18] = 4824 is not in [0, 4824)
	 [[{{node word_embedding_3/embedding_lookup}}]]

In [ ]:
from keras.backend import batch_get_value
import pickle

lstm.save("param","weights.h5","preprocessor")
symbolic_weights = getattr(lstm.model.optimizer, 'weights')
weight_values = batch_get_value(symbolic_weights)
with open('optimizer.pkl', 'wb') as f:
    pickle.dump(weight_values, f)


In [ ]:
from modelli.wrapper import Sequence

lstm=Sequence (word_embedding_dim=100, embeddings=model_2.wv)
lstm.fit(X_train, Y_train, X_val, Y_val, epochs=1)

In [ ]:
from modelli.models import save_model

save_model(lstm.model, "weights.h5","param")

In [ ]:
lstm.fit(X_train, Y_train, X_val, Y_val,epochs=10)